![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/Spark_NLP_Udemy_MOOC/Healthcare_NLP/DocumentMLClassifierModel.ipynb)

# **DocumentMLClassifierModel**

This notebook will cover the different parameters and usages of `DocumentMLClassifierModel`.

**🔗 Helpful Links:**
- Documentation : [DocumentMLClassifierModel](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#documentmlclassifier)

- Python Docs : [DocumentMLClassifierModel](https://nlp.johnsnowlabs.com/licensed/api/python/reference/autosummary/sparknlp_jsl/annotator/classification/document_ml_classifier/index.html#sparknlp_jsl.annotator.classification.document_ml_classifier.DocumentMLClassifierModel)

- Scala Docs : [DocumentMLClassifierModel](https://nlp.johnsnowlabs.com/licensed/api/com/johnsnowlabs/nlp/annotators/classification/DocumentMLClassifierModel.html)

- For extended examples of usage, see the [Spark NLP Workshop repository](https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/healthcare-nlp).

## **🎬 Colab Setup**

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.2/265.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.2/676.2 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.4 M

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

Please Upload your John Snow Labs License using the button below


Saving spark_nlp_for_healthcare_spark_ocr_8734.json to spark_nlp_for_healthcare_spark_ocr_8734.json


In [ ]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.settings.enforce_versions=False
nlp.install(refresh_install=True)

<IPython.core.display.Javascript object>

In [ ]:
# # Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_8734.json
👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.3.2, 💊Spark-Healthcare==5.3.3, running on ⚡ PySpark==3.4.0


In [ ]:
spark

## **🖨️ Input/Output Annotation Types**

- Input: `TOKEN`

- Output: `CATEGORY`

## **🔎 Parameters**


- `labels`: (list) Sets the name of labels to be used.

- `minTokenNgram`: (int) Sets minimum number of tokens for Ngrams.*

- `maxTokenNgram`: (int) Sets maximum number of tokens for Ngrams.*


> **\* Use with caution, as pretrained models were trained with specific values for minimum and maximum values of n-grams.**

## Prepare data

In [ ]:
data = spark.createDataFrame([
    ["I feel great after taking tylenol."],
    ["Toxic epidermal necrolysis resulted after 19 days of treatment with 5-fluorocytosine and amphotericin B."]
]).toDF("text")

In [ ]:
data.show(truncate=False)

+--------------------------------------------------------------------------------------------------------+
|text                                                                                                    |
+--------------------------------------------------------------------------------------------------------+
|I feel great after taking tylenol.                                                                      |
|Toxic epidermal necrolysis resulted after 19 days of treatment with 5-fluorocytosine and amphotericin B.|
+--------------------------------------------------------------------------------------------------------+



### `setLabels()`


The labels of the pretrained model `classifierml_ade` are `"False"` and `"True"` to determine if there were any Adverse Drug Event (ADE) on the text. Let's change the labels to `"Not ADE"` and `"ADE"`:

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer()\
    .setInputCols("document")\
    .setOutputCol("token")

classifier_ml = medical.DocumentMLClassifierModel.pretrained("classifierml_ade", "en", "clinical/models")\
    .setInputCols("token")\
    .setOutputCol("prediction")\
    .setLabels(["Not ADE", "ADE"])

clf_Pipeline = nlp.Pipeline(stages=[
    document_assembler,
    tokenizer,
    classifier_ml])


classifierml_ade download started this may take some time.
[OK!]


In [ ]:
clf_model = clf_Pipeline.fit(data)
result = clf_model.transform(data)

In [ ]:
result.select('text','prediction.result').show(truncate=False)

+--------------------------------------------------------------------------------------------------------+---------+
|text                                                                                                    |result   |
+--------------------------------------------------------------------------------------------------------+---------+
|I feel great after taking tylenol.                                                                      |[Not ADE]|
|Toxic epidermal necrolysis resulted after 19 days of treatment with 5-fluorocytosine and amphotericin B.|[ADE]    |
+--------------------------------------------------------------------------------------------------------+---------+



### `setMinTokenNgram()` and `setMaxTokenNgram()`

Defines the range of tokens to be used by the vectorizer model.


> **\* Use with caution, as pretrained models were trained with specific values for minimum and maximum values of n-grams.**


These parameters are used internally to create the features to be used as input to the model.

In [ ]:
# Change min to 2 and max to 4

classifier_ml.setMinTokenNgram(2).setMaxTokenNgram(4)

clf_Pipeline = nlp.Pipeline(stages=[
    document_assembler,
    tokenizer,
    classifier_ml])

clf_model = clf_Pipeline.fit(data)
result = clf_model.transform(data)

In [ ]:
result.select('text', 'prediction.result').show(truncate=False)

+--------------------------------------------------------------------------------------------------------+---------+
|text                                                                                                    |result   |
+--------------------------------------------------------------------------------------------------------+---------+
|I feel great after taking tylenol.                                                                      |[Not ADE]|
|Toxic epidermal necrolysis resulted after 19 days of treatment with 5-fluorocytosine and amphotericin B.|[ADE]    |
+--------------------------------------------------------------------------------------------------------+---------+



Please note that this specific pretrained model was fitted with (1,2) for the minimum and maxmum number of grams. Using different values will make the vectorizer model consider them as Out-of-Vocabulary (OOV). Usualy, it is recommended to use a subinterval of the pretrained model. This way all the tokens will be known by the model.

For example, let's use (1,1):

In [ ]:
classifier_ml.setMinTokenNgram(1).setMaxTokenNgram(1)

clf_Pipeline = nlp.Pipeline(stages=[
    document_assembler,
    tokenizer,
    classifier_ml])

clf_model = clf_Pipeline.fit(data)
result = clf_model.transform(data)
result.select('text', 'prediction.result').show(truncate=False)

+--------------------------------------------------------------------------------------------------------+---------+
|text                                                                                                    |result   |
+--------------------------------------------------------------------------------------------------------+---------+
|I feel great after taking tylenol.                                                                      |[Not ADE]|
|Toxic epidermal necrolysis resulted after 19 days of treatment with 5-fluorocytosine and amphotericin B.|[Not ADE]|
+--------------------------------------------------------------------------------------------------------+---------+



We can see that information present in bigrams was lost and the model now incorrectly classified the second document as `Not ADE`.